Install Snowpark if not already \
python -m pip install snowflake-snowpark-python

In [1]:
#import necessary libraries
from snowflake.snowpark.session import Session
from snowflake.snowpark.session import col
import snowflake.snowpark.session as f
import pandas as pd
from snowflake.snowpark.functions import lit
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import DecimalType, IntegerType
from snowflake.snowpark.exceptions import SnowparkSQLException, SnowparkJoinException
import time

Session allows us to create an instance of the snowflake and connect to the platform 

In [ ]:
start_time = time.time()

In [2]:
#establishing connection to the snowflake
connection_parameters = {"account": "kinesso.us-east-1",
                         "user": "pragatheesan.sreenivasan@initiative.com",
                         "authenticator": "externalbrowser",
                         "role": "INI_AGENCY_US",
                         "warehouse": "INI_AMAZON_US",
                         "database": "GR_KINESSO",
                         "schema":"INI_AMAZON_US"}

session = Session.builder.configs(connection_parameters).create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/d026e4c1-5892-497a-b9da-ee493c9f0364/saml2?SAMLRequest=lZJfb9owFMW%2FSuQ9J3ZC%2BBMLqCiIjaorCOg68WYSBywcO%2FN1CPDpZ0KRuodWmuQHyz7H%2Ft17bv%2FhVEjvyA0IrQYoDAjyuEp1JtRugF7XU7%2BHPLBMZUxqxQfozAE9DPvAClnSUWX3asn%2FVBys5x5SQJuLAaqMopqBAKpYwYHalK5GP59pFBBaGm11qiX6YPnawQC4sY7wbslAOLy9tSXFuK7roG4F2uxwRAjBJMFOdZV8u%2BtPrqZP9CEm8VXvFE6%2BeGd7FOrWgq%2BwtjcR0B%2Fr9cJfzFdr5I3uqGOtoCq4WXFzFCl%2FXT7fAMARHITiADqowOcMrB8GoHSdS3bgqS7KyrpXA7fDOc%2Bw1DvhCp9NBqg8iOzy8rsdPbWX3c3b8%2Fz7fPnWaW3K%2Fe58nMrJ9rF%2BSi7j0yUki04ep8j7dU82uiY7A6j4TF3ztO6IRLFPIrfWYUzbXdqOA9LrbZA3cXkKxWzjvEM3HEEhUqNB51Yr6apoKDMSdXichn67l0R%2BnHSZv00y5nMeJ600yUmrE%2BNryhG6TQ5tQMzwf%2FvRxx%2Fd70P44nKZTRZaivTsTbUpmP08tjAImxOR%2BXkjpbxgQo6yzDgCF5%2BUuh4bzqybdWsqjv

We can also establish structure type while declaring/creating a dataframe table and schema using StructType([StructField("column_name", DataType()).........])

In [ ]:
#storing table name
table_name = "AMZ_002_OUTPUT_ALL"

#extracting data from database using sql
query = session.sql(f"Select * from {table_name}")
query.show()

Collect function uses the extracted record to be stored as array of rows, in other words as list

In [ ]:
#extracting data from database directly using table
df = session.table(table_name).collect()
df

In [ ]:
#converting list to pandas df after list operations if necessary
fromList_df = pd.DataFrame(df)
fromList_df

In [ ]:
#filtering
dataframe = query.filter((col("AMAZON_LOB") == 'GCA') & (col("COUNTRY") != 'US'))
dataframe.show()
dataframe.columns

if session.sql("....") is used it can only be a select statement

In [3]:
#adding linkedin table to create a simple view and join two tables that is structured in snowflake
LinkedIn_table = "ADV_LINKEDIN"
LinkedIn_table_df = session.sql(f"Select * from {LinkedIn_table}")
LinkedIn_table_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Commented it out as it will create a new view in the sf db \
We can also create a stream from the view (an object to track all DML operations, to see the changes happen in the view)
- session.sql('CREATE or REPLACE STREAM 'STREAM_NAME' ON VIEW 'VIEW_NAME').collect()
- where all the changes will be stored in a list

In [ ]:
#creating a view of the table
#LinkedIn_table_df.create_or_replace_view('linkedIn_table_view')

In [4]:
#Selecting the required columns using select function and passing on the list of columns

linkedin_columns = ['STARTDATE','COUNTRY','CAMPAIGN_GROUPNAME','CAMPAIGN_GROUPID','CAMPAIGNNAME','CAMPAIGNID','CREATIVEID', 'CREATIVENAME','IMPRESSIONS','CLICKS',\
    'ACCOUNTCURRENCY','COSTINLOCALCURRENCY','VIDEOVIEWS','VIDEOSTARTS','VIDEOFIRSTQUARTILECOMPLETIONS',\
        'VIDEOMIDPOINTCOMPLETIONS','VIDEOTHIRDQUARTILECOMPLETIONS', 'VIDEOCOMPLETIONS', 'TOTALENGAGEMENTS','CHANNEL','CHANNEL_NAME','AMAZON_LOB']

l_df = LinkedIn_table_df.select(linkedin_columns)
l_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"STARTDATE"  |"COUNTRY"  |"CAMPAIGN_GROUPNAME"         |"CAMPAIGN_GROUPID"  |"CAMPAIGNNAME"                         |"CAMPAIGNID"  |"CREATIVEID"  |"CREATIVENAME"                                      |"IMPRESSIONS"  |"CLICKS"  |"ACCOUNTCURRENCY"  |"COSTINLOCALCURRENCY"  |"VIDEOVIEWS"  |"VIDEOSTARTS"  |"VIDEOFIRSTQUARTILECOMPLETIONS"  |"VIDEOMIDPOINTCOMPLETIONS"  |"VIDEOTHIRDQUARTILECOMPLETIONS"  |"VIDEOCOMPLETIONS"  |"TOTALENGAGEMENTS"  |"CHANNEL"  |"CHANNEL_NAME"  |"AMAZON_LOB"  |
--------------------------

In [5]:
#typecasting and renaming the desired columns 
l_df = l_df\
    .select(
        l_df['STARTDATE'].alias('DATE'),
        l_df['COUNTRY'].alias('COUNTRY'),
        l_df['CAMPAIGN_GROUPNAME'].cast('string').alias('CAMPAIGN_NAME'),
        l_df['CAMPAIGN_GROUPID'].cast('int').alias('CAMPAIGN_ID'),
        l_df['CAMPAIGNNAME'].cast('string').alias('PLACEMENT_NAME'),
        l_df['CAMPAIGNID'].cast('int').alias('PLACEMENT_ID'),
        l_df['CREATIVEID'].cast('string').alias('AD_ID'),
        l_df['CREATIVENAME'].alias('AD_NAME'),
        l_df['IMPRESSIONS'].cast('int').alias('IMPRESSIONS'),
        l_df['CLICKS'].cast('int').alias('CLICKS'),
        l_df['ACCOUNTCURRENCY'].cast('string').alias('CURRENCY'),
        l_df['COSTINLOCALCURRENCY'].cast('float').alias('NET_SPEND'),
        l_df['VIDEOVIEWS'].alias('VIDEO_VIEWS'),
        l_df['VIDEOSTARTS'].alias('VIDEO_STARTS'),
        l_df['VIDEOFIRSTQUARTILECOMPLETIONS'].alias('VIDEO_25'),
        l_df['VIDEOMIDPOINTCOMPLETIONS'].alias('VIDEO_50'),
        l_df['VIDEOTHIRDQUARTILECOMPLETIONS'].alias('VIDEO_75'),
        l_df['VIDEOCOMPLETIONS'].alias('VIDEO_100'),
        l_df['TOTALENGAGEMENTS'].alias('ENGAGEMENTS'),
        l_df['CHANNEL'].alias('CHANNEL'),
        l_df['CHANNEL_NAME'].alias('PLATFORM'),
        l_df['AMAZON_LOB'].alias('AMAZON_LOB')
        )
l_df.columns

['DATE',
 'COUNTRY',
 'CAMPAIGN_NAME',
 'CAMPAIGN_ID',
 'PLACEMENT_NAME',
 'PLACEMENT_ID',
 'AD_ID',
 'AD_NAME',
 'IMPRESSIONS',
 'CLICKS',
 'CURRENCY',
 'NET_SPEND',
 'VIDEO_VIEWS',
 'VIDEO_STARTS',
 'VIDEO_25',
 'VIDEO_50',
 'VIDEO_75',
 'VIDEO_100',
 'ENGAGEMENTS',
 'CHANNEL',
 'PLATFORM',
 'AMAZON_LOB']

In [6]:
#grouping the columns for aggregating spend, clicks...
l_df = l_df \
    .groupBy('DATE', 'COUNTRY', 'CAMPAIGN_NAME', 'CAMPAIGN_ID', 'PLACEMENT_NAME', 'PLACEMENT_ID', 'AD_ID', 'AD_NAME', 'CURRENCY', 'CHANNEL', 'PLATFORM', 'AMAZON_LOB') \
    .agg(
        F.sum('IMPRESSIONS').alias('IMPRESSIONS'),
        F.sum('CLICKS').alias('CLICKS'),
        F.sum('NET_SPEND').alias('NET_SPEND'),
        F.sum('VIDEO_VIEWS').alias('VIDEO_VIEWS'),
        F.sum('VIDEO_STARTS').alias('VIDEO_STARTS'),
        F.sum('VIDEO_25').alias('VIDEO_25'),
        F.sum('VIDEO_50').alias('VIDEO_50'),
        F.sum('VIDEO_75').alias('VIDEO_75'),
        F.sum('VIDEO_100').alias('VIDEO_100'),
        F.sum('ENGAGEMENTS').alias('ENGAGEMENTS'),
    )
l_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATE"      |"COUNTRY"  |"CAMPAIGN_NAME"              |"CAMPAIGN_ID"  |"PLACEMENT_NAME"                       |"PLACEMENT_ID"  |"AD_ID"    |"AD_NAME"                                           |"CURRENCY"  |"CHANNEL"  |"PLATFORM"  |"AMAZON_LOB"  |"IMPRESSIONS"  |"CLICKS"  |"NET_SPEND"  |"VIDEO_VIEWS"  |"VIDEO_STARTS"  |"VIDEO_25"  |"VIDEO_50"  |"VIDEO_75"  |"VIDEO_100"  |"ENGAGEMENTS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
# Loading meta table in
meta_table_name = 'ADV_META'
meta_df = session.sql(f'Select * from {meta_table_name}')
meta_df.show(10)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
#Selecting the required columns using select function and passing on the list of columns

meta_columns = ['DATE_START','COUNTRY','CAMPAIGN_NAME','CAMPAIGN_ID','ADSET_NAME','ADSET_ID','AD_ID','AD_NAME','IMPRESSIONS','INLINE_LINK_CLICKS',\
    'ACCOUNT_CURRENCY','SPEND','VIDEO_PLAY_ACTIONSVIDEO_VIEW','VIDEO_P25_WATCHED_ACTIONSVIDEO_VIEW','VIDEO_P50_WATCHED_ACTIONSVIDEO_VIEW',\
        'VIDEO_P75_WATCHED_ACTIONSVIDEO_VIEW','VIDEO_P100_WATCHED_ACTIONSVIDEO_VIEW', 'ACTIONSPOST_ENGAGEMENT','CHANNEL','CHANNEL_NAME','AMAZON_LOB']

m_df = meta_df.select(meta_columns)
m_df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATE_START"  |"COUNTRY"  |"CAMPAIGN_NAME"                                     |"CAMPAIGN_ID"      |"ADSET_NAME"                                        |"ADSET_ID"         |"AD_ID"            |"AD_NAME"                                           |"IMPRESSIONS"  |"INLINE_LINK_CLICKS"  |"ACCOUNT_CURRENCY"  |"SPEND"   |"VIDEO_PLAY_ACTIONSVIDEO_VIEW"  |"VIDEO_P25_WATCHED_ACTIONSVIDEO_VIEW"  |"VIDEO_P50_WATCHED_ACTION

In [9]:
#typecasting and renaming the desired columns 
m_df = m_df\
    .select(
        m_df['DATE_START'].alias('DATE'),
        m_df['COUNTRY'].alias('COUNTRY'),
        m_df['CAMPAIGN_NAME'].cast('string').alias('CAMPAIGN_NAME'),
        m_df['CAMPAIGN_ID'].cast('int').alias('CAMPAIGN_ID'),
        m_df['ADSET_NAME'].cast('string').alias('PLACEMENT_NAME'),
        m_df['ADSET_ID'].cast('int').alias('PLACEMENT_ID'),
        m_df['AD_ID'].cast('string').alias('AD_ID'),
        m_df['AD_NAME'].alias('AD_NAME'),
        m_df['IMPRESSIONS'].cast('int').alias('IMPRESSIONS'),
        m_df['INLINE_LINK_CLICKS'].cast('int').alias('CLICKS'),
        m_df['ACCOUNT_CURRENCY'].cast('string').alias('CURRENCY'),
        m_df['SPEND'].cast('float').alias('NET_SPEND'),
        m_df['VIDEO_PLAY_ACTIONSVIDEO_VIEW'].alias('VIDEO_VIEWS'),
        m_df['VIDEO_PLAY_ACTIONSVIDEO_VIEW'].alias('VIDEO_STARTS'),
        m_df['VIDEO_P25_WATCHED_ACTIONSVIDEO_VIEW'].alias('VIDEO_25'),
        m_df['VIDEO_P50_WATCHED_ACTIONSVIDEO_VIEW'].alias('VIDEO_50'),
        m_df['VIDEO_P75_WATCHED_ACTIONSVIDEO_VIEW'].alias('VIDEO_75'),
        m_df['VIDEO_P100_WATCHED_ACTIONSVIDEO_VIEW'].alias('VIDEO_100'),
        m_df['ACTIONSPOST_ENGAGEMENT'].alias('ENGAGEMENTS'),
        m_df['CHANNEL'].alias('CHANNEL'),
        m_df['CHANNEL_NAME'].alias('PLATFORM'),
        m_df['AMAZON_LOB'].alias('AMAZON_LOB')
        )
m_df.columns

['DATE',
 'COUNTRY',
 'CAMPAIGN_NAME',
 'CAMPAIGN_ID',
 'PLACEMENT_NAME',
 'PLACEMENT_ID',
 'AD_ID',
 'AD_NAME',
 'IMPRESSIONS',
 'CLICKS',
 'CURRENCY',
 'NET_SPEND',
 'VIDEO_VIEWS',
 'VIDEO_STARTS',
 'VIDEO_25',
 'VIDEO_50',
 'VIDEO_75',
 'VIDEO_100',
 'ENGAGEMENTS',
 'CHANNEL',
 'PLATFORM',
 'AMAZON_LOB']

In [12]:
#grouping the columns for aggregating spend, clicks...
m_df = m_df \
    .groupBy('DATE', 'COUNTRY', 'CAMPAIGN_NAME', 'CAMPAIGN_ID', 'PLACEMENT_NAME', 'PLACEMENT_ID', 'AD_ID', 'AD_NAME', 'CURRENCY', 'CHANNEL', 'PLATFORM', 'AMAZON_LOB') \
    .agg(
        F.sum('IMPRESSIONS').alias('IMPRESSIONS'),
        F.sum('CLICKS').alias('CLICKS'),
        F.sum('NET_SPEND').alias('NET_SPEND'),
        F.sum('VIDEO_VIEWS').alias('VIDEO_VIEWS'),
        F.sum('VIDEO_STARTS').alias('VIDEO_STARTS'),
        F.sum('VIDEO_25').alias('VIDEO_25'),
        F.sum('VIDEO_50').alias('VIDEO_50'),
        F.sum('VIDEO_75').alias('VIDEO_75'),
        F.sum('VIDEO_100').alias('VIDEO_100'),
        F.sum('ENGAGEMENTS').alias('ENGAGEMENTS'),
    )
m_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATE"      |"COUNTRY"  |"CAMPAIGN_NAME"                                     |"CAMPAIGN_ID"      |"PLACEMENT_NAME"                                    |"PLACEMENT_ID"     |"AD_ID"            |"AD_NAME"                                           |"CURRENCY"  |"CHANNEL"  |"PLATFORM"  |"AMAZON_LOB"  |"IMPRESSIONS"  |"CLICKS"  |"NET_SPEND"         |"VIDEO_VIEWS"  |"VIDEO_STARTS"  |"VIDEO_25"  |"VIDEO_50"  |"VIDEO_75"  |"VIDEO_100"  |"ENGAGEMENTS"  |
------------------------------------------------------------------------------------------------------

Can use 'join' functions matching column 'key' name with the other; like pandas lib to join two or more tables

In [13]:
#taking unionall to combine both tables
try:
    final_df = l_df.unionAll(m_df)
    final_df.show()
except Exception as e:
    print(e.message)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATE"      |"COUNTRY"  |"CAMPAIGN_NAME"              |"CAMPAIGN_ID"  |"PLACEMENT_NAME"                       |"PLACEMENT_ID"  |"AD_ID"    |"AD_NAME"                                           |"CURRENCY"  |"CHANNEL"  |"PLATFORM"  |"AMAZON_LOB"  |"IMPRESSIONS"  |"CLICKS"  |"NET_SPEND"  |"VIDEO_VIEWS"  |"VIDEO_STARTS"  |"VIDEO_25"  |"VIDEO_50"  |"VIDEO_75"  |"VIDEO_100"  |"ENGAGEMENTS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
#applying filter as GCA and not US
final_df = final_df.filter((F.col('AMAZON_LOB') == 'GCA') & (F.col('COUNTRY') != 'US'))
final_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATE"      |"COUNTRY"  |"CAMPAIGN_NAME"                                     |"CAMPAIGN_ID"  |"PLACEMENT_NAME"                                    |"PLACEMENT_ID"  |"AD_ID"    |"AD_NAME"                                           |"CURRENCY"  |"CHANNEL"  |"PLATFORM"  |"AMAZON_LOB"  |"IMPRESSIONS"  |"CLICKS"  |"NET_SPEND"  |"VIDEO_VIEWS"  |"VIDEO_STARTS"  |"VIDEO_25"  |"VIDEO_50"  |"VIDEO_75"  |"VIDEO_100"  |"ENGAGEMENTS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
#Using group_by function for aggregating spend. Aggregating functions can accessed after importing snowpark.function lib

grouped_spend = final_df.group_by(['DATE','COUNTRY','CAMPAIGN_NAME']).agg([F.sum('NET_SPEND').alias('SPEND')])
grouped_spend.show()

----------------------------------------------------------------------------------------------------
|"DATE"      |"COUNTRY"  |"CAMPAIGN_NAME"                                     |"SPEND"             |
----------------------------------------------------------------------------------------------------
|2024-01-29  |DE         |2024-01_deu_gsmc_vv_vv_linkin_auc_amazon_gsmc_s...  |2920.5499000000004  |
|2023-06-19  |BE         |2023-05_be_gca_awe_aw_linkin_fix_amazon_gca_soc...  |2471.4498           |
|2023-05-23  |BE         |2023-05_be_gca_awe_aw_linkin_fix_amazon_gca_soc...  |2650.6              |
|2023-06-09  |BE         |2023-05_be_gca_awe_aw_linkin_fix_amazon_gca_soc...  |2356.6601           |
|2023-05-12  |UK         |2023-04_gbr_gca_awe_aw_linkin_auc_amazon_gca_so...  |1522.8799           |
|2023-07-06  |UK         |2023-04_gbr_gca_awe_aw_linkin_auc_amazon_gca_so...  |15173.8601          |
|2023-11-22  |UK         |2023-09_gbr_rmi-lan_awe_aw_linkin_fix_amazon_gc...  |833.26      

In [16]:
#Calculating CTR, using the snowpark df
final_df = final_df.withColumn("CTR",(final_df["CLICKS"]/final_df["IMPRESSIONS"])*100)
final_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATE"      |"COUNTRY"  |"CAMPAIGN_NAME"                                     |"CAMPAIGN_ID"  |"PLACEMENT_NAME"                                    |"PLACEMENT_ID"  |"AD_ID"    |"AD_NAME"                                           |"CURRENCY"  |"CHANNEL"  |"PLATFORM"  |"AMAZON_LOB"  |"IMPRESSIONS"  |"CLICKS"  |"NET_SPEND"  |"VIDEO_VIEWS"  |"VIDEO_STARTS"  |"VIDEO_25"  |"VIDEO_50"  |"VIDEO_75"  |"VIDEO_100"  |"ENGAGEMENTS"  |"CTR"     |
----------------------------------------------------------------------------------------------------------------------------

In [19]:
#closing the session with sf
session.close()

In [ ]:
print("---%s seconds ---"%(time.time()-start_time))